##### Step 1 - Run "configuration" Notebook

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date", "2021-04-18")
v_file_date = dbutils.widgets.get("p_file_date")

##### Step 2 - Read required files using Dataframe and rename fields as required

In [0]:
races_df = spark.read.format("delta").load(f"{processed_folder_path}/races") \
    .withColumnRenamed("name", "race_name") \
    .withColumnRenamed("race_timestamp", "race_date")

In [0]:
# display(races_df.limit(5))

race_id,race_year,round,circuit_id,race_name,race_date,ingestion_date,data_source,file_date
1,2009,1,1,Australian Grand Prix,2009-03-29T06:00:00Z,2023-12-22T11:00:50.106Z,Ergast,2021-04-18
2,2009,2,2,Malaysian Grand Prix,2009-04-05T09:00:00Z,2023-12-22T11:00:50.106Z,Ergast,2021-04-18
3,2009,3,17,Chinese Grand Prix,2009-04-19T07:00:00Z,2023-12-22T11:00:50.106Z,Ergast,2021-04-18
4,2009,4,3,Bahrain Grand Prix,2009-04-26T12:00:00Z,2023-12-22T11:00:50.106Z,Ergast,2021-04-18
5,2009,5,4,Spanish Grand Prix,2009-05-10T12:00:00Z,2023-12-22T11:00:50.106Z,Ergast,2021-04-18


In [0]:
circuits_df = spark.read.format("delta").load(f"{processed_folder_path}/circuits") \
    .withColumnRenamed("location", "circuit_location")

In [0]:
# display(circuits_df.limit(5))

circuit_id,circuit_ref,name,circuit_location,country,latitude,longitude,altitude,ingestion_date,data_source,file_date
1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,2023-12-22T11:00:51.502Z,Ergast,2021-04-18
2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.738,18,2023-12-22T11:00:51.502Z,Ergast,2021-04-18
3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,7,2023-12-22T11:00:51.502Z,Ergast,2021-04-18
4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57,2.26111,109,2023-12-22T11:00:51.502Z,Ergast,2021-04-18
5,istanbul,Istanbul Park,Istanbul,Turkey,40.9517,29.405,130,2023-12-22T11:00:51.502Z,Ergast,2021-04-18


In [0]:
drivers_df = spark.read.format("delta").load(f"{processed_folder_path}/drivers") \
    .withColumnRenamed("name", "driver_name") \
    .withColumnRenamed("number", "driver_number") \
    .withColumnRenamed("nationality", "driver_nationality")

In [0]:
# display(drivers_df.limit(5))

driver_id,driver_ref,driver_number,code,driver_name,dob,driver_nationality,ingestion_date,data_source,file_date
1,hamilton,44,HAM,Lewis Hamilton,1985-01-07,British,2023-12-22T11:00:51.217Z,Ergast,2021-04-18
2,heidfeld,null,HEI,Nick Heidfeld,1977-05-10,German,2023-12-22T11:00:51.217Z,Ergast,2021-04-18
3,rosberg,6,ROS,Nico Rosberg,1985-06-27,German,2023-12-22T11:00:51.217Z,Ergast,2021-04-18
4,alonso,14,ALO,Fernando Alonso,1981-07-29,Spanish,2023-12-22T11:00:51.217Z,Ergast,2021-04-18
5,kovalainen,null,KOV,Heikki Kovalainen,1981-10-19,Finnish,2023-12-22T11:00:51.217Z,Ergast,2021-04-18


In [0]:
constructors_df = spark.read.format("delta").load(f"{processed_folder_path}/constructors") \
    .withColumnRenamed("name", "team")

In [0]:
# display(constructors_df.limit(5))

constructor_id,constructor_ref,team,nationality,ingestion_date,data_source,file_date
1,mclaren,McLaren,British,2023-12-22T11:00:50.346Z,Ergast,2021-04-18
2,bmw_sauber,BMW Sauber,German,2023-12-22T11:00:50.346Z,Ergast,2021-04-18
3,williams,Williams,British,2023-12-22T11:00:50.346Z,Ergast,2021-04-18
4,renault,Renault,French,2023-12-22T11:00:50.346Z,Ergast,2021-04-18
5,toro_rosso,Toro Rosso,Italian,2023-12-22T11:00:50.346Z,Ergast,2021-04-18


In [0]:
results_df = spark.read.format("delta").load(f"{processed_folder_path}/results") \
    .filter(f"file_date = '{v_file_date}'") \
    .withColumnRenamed("time", "race_time") \
    .withColumnRenamed("race_id", "result_race_id") \
    .withColumnRenamed("file_date", "result_file_date")

In [0]:
# display(results_df.limit(5))

result_id,result_race_id,driver_id,constructor_id,number,grid,position,position_text,position_order,points,laps,race_time,milliseconds,fastest_lap,rank,fastest_lap_time,fastest_lap_speed,ingestion_date,data_source,result_file_date
21546,873,4,6,5,5,3,3,3,15.0,59,+15.227,7241371,53,5,1:52.709,162.034,2023-12-22T10:51:17.188Z,Ergast,2021-03-21
21558,873,5,207,20,19,15,15,15,0.0,59,+107.967,7334111,48,18,1:55.233,158.485,2023-12-22T10:51:17.188Z,Ergast,2021-03-21
21555,873,10,206,24,20,12,12,12,0.0,59,+91.918,7318062,30,21,1:56.057,157.36,2023-12-22T10:51:17.188Z,Ergast,2021-03-21
21564,873,30,131,7,9,null,R,21,0.0,38,\N,null,26,20,1:56.047,157.374,2023-12-22T10:51:17.188Z,Ergast,2021-03-21
21566,873,39,164,23,23,null,R,23,0.0,30,\N,null,26,24,1:58.507,154.107,2023-12-22T10:51:17.188Z,Ergast,2021-03-21


##### Step 3 - Join Races and Circuits dataframe

In [0]:
race_circuits_df = races_df.join(circuits_df, races_df.circuit_id == circuits_df.circuit_id) \
    .select(races_df.race_id, races_df.race_year, races_df.race_name, races_df.race_date, circuits_df.circuit_location)
    

##### Step 4 - Join results, drivers, constructors to the races_circuits_df

In [0]:
race_results_df = race_circuits_df.join(results_df, race_circuits_df.race_id == results_df.result_race_id) \
    .join(drivers_df, drivers_df.driver_id == results_df.driver_id) \
    .join(constructors_df, constructors_df.constructor_id == results_df.constructor_id)

In [0]:
from pyspark.sql.functions import current_timestamp

##### Step 5 - Select required fields

In [0]:
final_df = race_results_df.select("race_id", "race_year", "race_name", "race_date", "circuit_location", 
                                  "driver_name", "driver_number", "driver_nationality", "team",  "grid", "fastest_lap", "race_time", "points", "position", "result_file_date") \
    .withColumn("created_date", current_timestamp()) \
    .withColumnRenamed("result_file_date", "file_date")

##### Step 6 - Write final data to the presentation container

In [0]:
merge_condition = "tgt.driver_name = src.driver_name AND tgt.race_id = src.race_id"
merge_delta_deta(final_df, "f1_presentation", "race_results", presentation_folder_path, merge_condition, "race_id")

In [0]:
# %sql
# SELECT race_id, COUNT(1)
# FROM f1_presentation.race_results
# GROUP BY race_id
# ORDER BY race_id DESC
# LIMIT 5;

race_id,count(1)
1053,20
1052,20
1047,20
1046,20
1045,20


In [0]:
%sql
SELECT * FROM f1_presentation.race_results
LIMIT 5;

race_id,race_year,race_name,race_date,circuit_location,driver_name,driver_number,driver_nationality,team,grid,fastest_lap,race_time,points,position,file_date,created_date
496,1981,Canadian Grand Prix,null,Montreal,Michele Alboreto,null,Italian,Tyrrell,22,null,\N,0.0,11,2021-03-21,2023-12-22T10:54:04.402Z
496,1981,Canadian Grand Prix,null,Montreal,Andrea de Cesaris,null,Italian,McLaren,13,null,\N,0.0,null,2021-03-21,2023-12-22T10:54:04.402Z
496,1981,Canadian Grand Prix,null,Montreal,Alain Prost,null,French,Renault,4,null,\N,0.0,null,2021-03-21,2023-12-22T10:54:04.402Z
496,1981,Canadian Grand Prix,null,Montreal,Derek Warwick,null,British,Toleman,0,null,\N,0.0,null,2021-03-21,2023-12-22T10:54:04.402Z
496,1981,Canadian Grand Prix,null,Montreal,Riccardo Patrese,null,Italian,Arrows,18,null,\N,0.0,null,2021-03-21,2023-12-22T10:54:04.402Z


In [0]:
dbutils.notebook.exit("Success")